## **Implementasi Arsitektur Xception pada Deep Learning CNN untuk Klasifikasi Citra Kanker Kulit**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
import numpy as np
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from keras.utils import plot_model

In [ ]:
# Fungsi untuk Load Data dengan Label yang Konsisten
def load_data(data_directory):
    images = []
    labels = []

    # Ambil daftar kelas dan urutkan agar sesuai dengan LabelEncoder
    class_names = sorted(os.listdir(data_directory))

    # Buat LabelEncoder dengan urutan kelas yang benar
    label_encoder = LabelEncoder()
    label_encoder.fit(class_names)

    for class_name in class_names:
        class_path = os.path.join(data_directory, class_name)

        if not os.path.isdir(class_path):
            continue  # Skip jika bukan direktori kelas

        for filename in os.listdir(class_path):
            filepath = os.path.join(class_path, filename)
            img = cv2.imread(filepath)

            if img is not None:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = cv2.resize(img, (224, 224))
                img = img / 255.0
                images.append(img)
                labels.append(class_name)  # Simpan sebagai string

    # Konversi label ke bentuk numerik dengan label_encoder
    labels = label_encoder.transform(labels)

    return np.array(images), np.array(labels), label_encoder

In [4]:
# Load Dataset
data_directory = 'kanker_kulit-2245'
images, labels, label_encoder = load_data(data_directory)

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(x=label_encoder.inverse_transform(labels))
plt.xlabel('Class')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.show()


In [ ]:
# Split Data (Train, Validation, Test) dengan stratifikasi
X_train, X_temp, y_train, y_temp = train_test_split(
    images, labels, test_size=0.2, random_state=42, stratify=labels
)

X_valid, X_test, y_valid, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

In [ ]:
print(f'Train set: {len(X_train)} samples')
print(f'Validation set: {len(X_valid)} samples')
print(f'Test set: {len(X_test)} samples')

In [ ]:
from collections import Counter

# Hitung jumlah sampel untuk setiap kelas
class_counts = Counter(labels)

# Tampilkan jumlah sampel dengan label yang benar
for class_label, count in class_counts.items():
    class_name = label_encoder.inverse_transform([class_label])[0]  # Pastikan label sesuai
    print(f"Kelas: {class_name} -> Jumlah sampel: {count}")

# Fungsi untuk menampilkan distribusi kelas dalam set data
def show_class_distribution(labels, label_encoder, set_name):
    class_counts = Counter(labels)
    print(f"\nDistribusi Kelas pada {set_name} Set:")
    for class_label, count in sorted(class_counts.items()):
        class_name = label_encoder.inverse_transform([class_label])[0]
        print(f"Kelas: {class_name} -> Jumlah sampel: {count}")

# Tampilkan distribusi kelas untuk masing-masing set
show_class_distribution(y_train, label_encoder, "Train")
show_class_distribution(y_valid, label_encoder, "Validation")
show_class_distribution(y_test, label_encoder, "Test")

In [ ]:
fig = plt.figure(figsize=(20,5))

for i in range(30):
    ax = fig.add_subplot(3,10, i+1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(X_train[i]))

In [ ]:
# Konversi label ke one-hot encoding
y_train = to_categorical(y_train, num_classes=len(label_encoder.classes_))
y_valid = to_categorical(y_valid, num_classes=len(label_encoder.classes_))
y_test = to_categorical(y_test, num_classes=len(label_encoder.classes_))

In [ ]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator()

# Generator Data
train_generator = train_datagen.flow(X_train, y_train, batch_size=32)
val_generator = val_datagen.flow(X_valid, y_valid, batch_size=32)

In [ ]:
# Load model Xception tanpa fully connected layers (top=False)
base_model = Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Buka beberapa layer terakhir agar bisa di-train
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Bangun model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu', kernel_regularizer='l2'),  # Tambahkan Regularization
    Dropout(0.4),  # Dropout lebih besar
    Dense(len(label_encoder.classes_), activation='softmax')
])
# Menampilkan arsitektur model
model.summary()

In [ ]:
plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)

In [ ]:
# Callback untuk menghentikan training berdasarkan loss atau akurasi
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('loss') < 0.5 and logs.get('val_loss') < 0.8) or \
           (logs.get('accuracy') >= 0.90 and logs.get('val_accuracy') >= 0.80):
            print(f"\nStopping training at epoch {epoch+1} as conditions met")
            self.model.stop_training = True

callbacks = [
    myCallback(),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1)
]

In [ ]:
# Pastikan model dikompilasi sebelum training
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=0.0005),
    metrics=['accuracy']
)

# Training Model dengan class weight yang benar
history = model.fit(
    train_generator,
    epochs=100,
    validation_data=val_generator,
    callbacks=callbacks,
)

In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

best_epoch = val_accuracy.index(max(val_accuracy)) + 1  # Add 1 because epochs are 1-indexed

print(f"Best Accuracy: {accuracy[best_epoch-1]:.4f} at Epoch {best_epoch}")
print(f"Best Validation Accuracy: {val_accuracy[best_epoch-1]:.4f} at Epoch {best_epoch}")

In [ ]:
y_hat_with_cnn = model.predict(X_test)
test_with_cnn = model.evaluate(X_test, y_test)
print('Test Loss = ', test_with_cnn[0], 'Test Accuracy = ', test_with_cnn[1])

In [ ]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=label_encoder.classes_,
            yticklabels=label_encoder.classes_)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

print('Classification Report:')
print(classification_report(y_true, y_pred_classes,
                            target_names=label_encoder.classes_))

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

print('Classification Report:')
print(classification_report(y_true, y_pred_classes, target_names=label_encoder.classes_))

In [ ]:
# Visualisasi hasil prediksi
classes = label_encoder.classes_
fig = plt.figure(figsize=(20, 8))
for i, idx in enumerate(np.random.choice(X_test.shape[0], size=16, replace=False)):
    ax = fig.add_subplot(4, 4, i + 1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(X_test[idx]))
    pred_idx = np.argmax(y_pred[idx])
    true_idx = np.argmax(y_test[idx])
    ax.set_title("{} ({})".format(classes[pred_idx], classes[true_idx]),
                 color=("green" if pred_idx == true_idx else "red"))

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
model.save('8981-remnant.h5')

In [ ]:
# Import Library yang Diperlukan
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from google.colab import files
import matplotlib.pyplot as plt

# Muat Model yang telah dilatih
model = tf.keras.models.load_model('/content/drive/MyDrive/model2.h5')

# Upload Foto
uploaded = files.upload()

# Definisikan kelas dan kategori medisnya
class_info = {
    'Actinic Keratosis (Solar Keratosis)': 'Pre-cancerous',
    'Basal Cell Carcinoma (Carcinoma Basocellulare)': 'Malignant',
    'Dermatofibroma (Histiofibroma)': 'Benign',
    'Kaposi Sarcoma (Sarcoma Kaposi)': 'Malignant',
    'Melanocytic Nevus (Naevus Melanocyticus)': 'Benign',
    'Melanoma (Melanoma Malignum)': 'Malignant',
    'Pigmented Benign Keratosis (Keratosis Seborrhoica Pigmentosa)': 'Benign',
    'Seborrheic Keratosis (Keratosis Seborrhoica)': 'Benign',
    'Solar Lentigo (Lentigo Senilis)': 'Benign',
    'Squamous Cell Carcinoma (Carcinoma Squamocellulare)': 'Malignant',
    'Vascular Lesions (Lesio Vascularis)': 'Varied (mostly benign, rarely malignant)'
}

# Muat dan prediksi gambar
for fn in uploaded.keys():
    img_path = fn
    img = image.load_img(img_path, target_size=(224, 224))
    plt.imshow(img)
    plt.axis('off')
    plt.show()

    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    predictions = model.predict(img_array)

    class_names = list(class_info.keys())
    predicted_class = class_names[np.argmax(predictions[0])]
    predicted_category = class_info[predicted_class]

    print(f'Prediksi: {predicted_class}')
    print(f'Kategori Medis: {predicted_category}')